In [1]:
import os
import platform

os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# import matplotlib.pyplot as plt
import tqdm
import cv2
import pandas as pd

from street_signs_identify.utils.x_image_loader import XImageLoader
import street_signs_identify.utils.check_overlap as co

from street_signs_identify.detector.craft_detector import CRAFTDetector

In [2]:
folder_name = "test"
data_path = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\x\\{folder_name}'
output_folder = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xc\\{folder_name}'
output_folder_gt = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xc_gt\\{folder_name}'
output_folder_filtered_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xc_filter\\{folder_name}'
output_folder_craft_pred = f'C:\\Users\\MAGIC\\Ching\\survey\\plan\\rawdata_all\\7types_v2\\xc_craft\\{folder_name}'

if platform.system() == "Linux":
    folder_name = "./dataset/"
    data_path = os.path.join(folder_name, "2Penghu")
    output_folder = os.path.join(folder_name, "cropped_2Penghu")
    output_folder_gt = os.path.join(output_folder, "ref_gt")
    output_folder_filtered_pred = os.path.join(output_folder, "filtered_pred")
    output_folder_craft_pred = os.path.join(output_folder, "craft_pred")


os.makedirs(output_folder, exist_ok=True)
os.makedirs(output_folder_gt, exist_ok=True)
os.makedirs(output_folder_filtered_pred, exist_ok=True)
os.makedirs(output_folder_craft_pred, exist_ok=True)

In [3]:
detector = CRAFTDetector()
x_image_loader = XImageLoader(data_path, annotation_file="labels.csv")

In [4]:
x_image_loader._info

,raw_filename,filename,category,box,label,polygon,x0,y0,x1,y1
0,221005_013854877_Camera_0.jpg,1Tainan_nan,1,"[(525, 838), (522, 892), (718, 879), (721, 825)]",沙崙綠能科技示範場域,"POLYGON ((525 838, 522 892, 718 879, 721 825, ...",522,825,721,892
1,221005_013854877_Camera_0.jpg,1Tainan_nan,1,"[(521, 893), (518, 947), (714, 934), (717, 880)]",大台南會展中心,"POLYGON ((521 893, 518 947, 714 934, 717 880, ...",518,880,717,947
2,221005_013854877_Camera_0.jpg,1Tainan_nan,1,"[(723, 826), (720, 880), (937, 869), (936, 814)]",資安暨智慧科技研發大樓,"POLYGON ((723 826, 720 880, 937 869, 936 814, ...",720,814,937,880
3,221005_013854877_Camera_0.jpg,1Tainan_nan,1,"[(528, 785), (525, 839), (721, 826), (724, 772)]",高鐵台南站,"POLYGON ((528 785, 525 839, 721 826, 724 772, ...",525,772,724,839
4,221005_013854877_Camera_0.jpg,1Tainan_nan,1,"[(721, 879), (718, 933), (935, 922), (934, 867)]",臺灣智駕測試實驗室,"POLYGON ((721 879, 718 933, 935 922, 934 867, ...",718,867,935,933
...,...,...,...,...,...,...,...,...,...,...
274,220707_060159380_Camera_1.jpg,3Taoyuan_nan,6,"[(2129, 819), (2138, 928), (2427, 891), (2413,...",號停車場,"POLYGON ((2129 819, 2138 928, 2427 891, 2413 7...",2129,767,2427,928
275,220707_053755977_Camera_1.jpg,3Taoyuan_nan,1,"[(2065, 917), (2067, 954), (1944, 956), (1943,...",號停車場,"POLYGON ((2065 917, 2067 954, 1944 956, 1943 9...",1943,916,2067,956
276,220707_053755977_Camera_1.jpg,3Taoyuan_nan,1,"[(2246, 963), (2248, 1005), (2451, 990), (2451...",出境大廳,"POLYGON ((2246 963, 2248 1005, 2451 990, 2451 ...",2246,942,2451,1005
277,220707_053755977_Camera_1.jpg,3Taoyuan_nan,1,"[(2251, 1050), (2451, 1040), (2451, 991), (224...",號停車場,"POLYGON ((2251 1050, 2451 1040, 2451 991, 2247...",2247,991,2451,1050


In [5]:
global_index = 0

label_df = pd.DataFrame(columns=["filename", "category", "words", "src_file", "gt_index", "detected_index", "IOU"])

for image_name in tqdm.tqdm(x_image_loader._image_names):
    gt_image = x_image_loader[image_name]
    detected_image = detector(gt_image.image)
    if detected_image is None:
        continue
    
    overlapping_info = co.check_overlapping_area(gt_image.info, detected_image.info)
    if overlapping_info is None:
        continue

    # save gt plot
    gt_img = gt_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_gt, image_name), cv2.cvtColor(gt_img, cv2.COLOR_RGB2BGR))
    
    # save craft plot
    det_img = detected_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_craft_pred, image_name), cv2.cvtColor(det_img, cv2.COLOR_RGB2BGR))

    filtered_image = detected_image.keeps_index(overlapping_info["index_2nd"])

    filtered_det_img = filtered_image.draw_all_box()
    cv2.imwrite(os.path.join(output_folder_filtered_pred, image_name), cv2.cvtColor(filtered_det_img, cv2.COLOR_RGB2BGR))

    for row_idx, row in overlapping_info.iterrows():
        gt_index, detected_index = row["index_1st"], row["index_2nd"]
        detected_instance = detected_image[detected_index]
        if not detected_instance.legal:
            continue
        
        label_df.loc[global_index, "filename"] = f"{global_index}.jpg"
        label_df.loc[global_index, "src_file"] = image_name
        label_df.loc[global_index, "category"] = gt_image.info.loc[gt_index, "category"]
        label_df.loc[global_index, "words"] = gt_image.info.loc[gt_index, "label"]
        label_df.loc[global_index, "IOU"] = row["IOU"]
        label_df.loc[global_index, "gt_index"] = int(gt_index)
        label_df.loc[global_index, "detected_index"] = int(detected_index)

        crapped_img = detected_instance.image
        cv2.imwrite(os.path.join(output_folder, f"{global_index}.jpg"), cv2.cvtColor(crapped_img, cv2.COLOR_RGB2BGR))
        global_index+=1

for src_file, dg in label_df.groupby("src_file"):
    for detected_idx, sub_dg in dg.groupby("detected_index"):
        sorted_index = sub_dg["IOU"].sort_index().index
        # print(sorted_index)
        label_df.loc[sorted_index, "priority"] = range(len(sorted_index))

# for src_file, dg in label_df.groupby("src_file"):
#     print(dg)

label_df.to_csv(os.path.join(output_folder, "labels.csv"), encoding = 'big5')

100%|████████████████████████████████████████████████████████████████████████████████| 140/140 [01:18<00:00,  1.78it/s]
